In [1]:
import random
import string
import re
import random
import string
import collections

class HangmanGuesser:
    def __init__(self, word_list):
        self.word_list = word_list
        self.new_dict= []
        self.guesses = []

    def make_guess(self, masked_word):
        # Clean the masked word by removing spaces and converting to lowercase
        cleaned_word = "".join(masked_word.split()).lower()     
        # Filter the word list based on the known letters
        new_dict=[]
        desired_length = len(cleaned_word)
        pattern_to_match=masked_word
        #if len(cleaned_word)-(masked_word.count('_')) >= 1:
        #    unigram_model = self.train_unigram_model(word_list)
        #    top_probability_words = self.get_top_probability_words(unigram_model, pattern_to_match)
        #    print(f"Top probability words based on the pattern '{pattern_to_match}': {top_probability_words}")
        #    self.wordlist = top_probability_words

        filtered_words = [word for word in self.word_list if self.matches_pattern(cleaned_word, word)]
        interpolated_words = self.interpolate_words_from_pattern(filtered_words, masked_word, 25)
        for e in interpolated_words:
            if len(e) >= len(cleaned_word) :
                new_dict.append(e)    
        new_dict.extend(filtered_words)
        #print("Number of correct guess : ",len(cleaned_word)-(masked_word.count('_')))
        
            
        # Choose the most frequent unguessed letter from the filtered words
        guess = self.get_best_guess(new_dict)
        
        # Add the guessed letter to the set of guesses        
        self.guesses.append(guess)
   
        print(guess)
        return guess

    def matches_pattern(self, pattern, word):
        # Check if the word matches the given pattern
        return all(c == w if c.isalpha() else True for c, w in zip(pattern, word))
    
    def interpolate_words_from_pattern(self,word_list, pattern, num_interpolations):
        #interpolated_words = []
        pattern_regex = re.compile("^" + pattern.replace('_', '.') + "$")
        #print(pattern," , ",pattern_regex)
        #matching_words = [word for word in word_list if pattern_regex.match(word)]
        matching_words = [word for word in word_list if re.match(pattern_regex,word)]
        interpolated_words = random.sample(matching_words, min(num_interpolations, len(matching_words)))

        return interpolated_words

    def get_best_guess(self, list_):
        # Count the occurrence of each letter in the filtered word list
        best_guess='!'
        letter_counts = {}
        for word in list_:
            for letter in word:
                if letter.isalpha() and letter not in self.guesses:
                    letter_counts[letter] = letter_counts.get(letter, 0) + 1

        # Sort letters by frequency in descending order
        sorted_letters = sorted(letter_counts.items(), key=lambda x: x[1], reverse=True)
        print(sorted_letters)
        # Choose the most frequent unguessed letter
        best_guess = next((letter for letter, _ in sorted_letters if letter not in self.guesses), '!')
        full_dictionary_common_letter_sorted = collections.Counter("".join(self.word_list)).most_common()
        if best_guess == '!' or best_guess == '':
            print(sorted_letters)
            sorted_letters = full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letters:
                if letter not in self.guesses:
                    best_guess = letter
                    break         
        #if best_guess=='!':
        #print("Word list assigned in get_best_guess() : ",word_list)

        return best_guess

    def train_unigram_model(self,corpus):
        unigram_model = collections.defaultdict(int)
        for sentence in corpus:
            words = sentence.split()
            for word in words:
                unigram_model[word] += 1

        total_words = sum(unigram_model.values())
        for word in unigram_model:
            unigram_model[word] /= total_words
        return dict(unigram_model)
    
    def get_top_probability_words(self,unigram_model, pattern, top_n=10):
        matched_words = [word for word in unigram_model.keys() if len(pattern) == len(word) and all(p == '_' or p.lower() == w.lower() for p, w in zip(pattern, word))]
        top_words = sorted(matched_words, key=lambda word: unigram_model[word], reverse=True)[:top_n]
        return top_words

# Example usage
#word_list = ["hangman", "python", "programming", "example", "strategy", "algorithm", "computer", "developer","only"]
#target=["mangoes","hangman", "python", "programming", "example", "strategy", "algorithm", "computer", "developer","only","pains"]
target_path="new_words.txt"
with open(target_path, 'r') as file:
    target= [line.strip() for line in file.readlines()]
file_path = 'words_250000_train.txt'
with open(file_path, 'r') as file:
    word_list = [line.strip() for line in file.readlines()]
guesser = HangmanGuesser(word_list)
total_practice_runs=10
total_practice_successes=0

for e in range(total_practice_runs):
    
    guesser.guesses.clear()
    print(" Run number : ", e+1 , " and success : ",total_practice_successes)
    # Simulate Hangman game
    #target_word="gaseous"
    target_word = random.choice(word_list)
    same=-1;
    for word in word_list:
        if target_word == word: 
            same=0
            target_word = random.choice(word_list)
            word=word_list[0]
        else:    
            same=1
            
    if same==0:
        print("Word is in list !!!!!!!!!")
        e=e+1
        continue
    
    else :
        print("Word is not in list !!!!!!!!!")
        
            
    #target_word = random.choice(word_list)
    masked_word = ''.join('_' if c.isalpha() else c for c in target_word)
    trial=6
    a=0
    print("========== target word = ",target_word,"=================")
    for _ in range(20):  # Assuming 10 guesses for demonstration
        a=a+1
        
        print(trial,",",a, " : Current masked word:", masked_word)
        guess = guesser.make_guess(masked_word)
        print("Guessing letter:", guess)

        # Simulate checking the guessed letter in the target word
        for i, c in enumerate(target_word):
            if c == guess:
                masked_word = masked_word[:i] + c + masked_word[i + 1:]
       
        guess_wrong=0
        for k in target_word:
            if k == guess:
                guess_wrong= -1
                #print("guess word: ",guess," and letter in target word: ",k)
                break
            else:
                #print("guess word: ",guess," and letter in target word: ",k)

                guess_wrong= 1
    
        if guess_wrong==1 :
            trial= trial - 1
        
        if trial==0:
            break
            
        if '_' not in masked_word:
            print(" Successfully Word guessed:", target_word)
            total_practice_successes=total_practice_successes+1
            break
    print(" Run number : ", e+1 , " and success : ",total_practice_successes)

    
print("End of game")


 Run number :  1  and success :  0
Word is not in list !!!!!!!!!
========== target word =  aminoacetone =================
6 , 1  : Current masked word: ____________
[('e', 233772), ('i', 184772), ('a', 179864), ('n', 152278), ('o', 150079), ('r', 149251), ('s', 148489), ('t', 137296), ('l', 122449), ('c', 89381), ('u', 77309), ('d', 74865), ('p', 65798), ('m', 62195), ('h', 58061), ('g', 51857), ('y', 40996), ('b', 39847), ('f', 26431), ('v', 21057), ('k', 18686), ('w', 17734), ('z', 8749), ('x', 6053), ('q', 3987), ('j', 3790)]
e
Guessing letter: e
6 , 2  : Current masked word: _______e___e
[('a', 48703), ('s', 37820), ('i', 37172), ('r', 36869), ('n', 32386), ('o', 32178), ('l', 29642), ('t', 29473), ('d', 21340), ('u', 19694), ('c', 19220), ('m', 16122), ('p', 14491), ('h', 14129), ('g', 12693), ('b', 12189), ('y', 10184), ('k', 7953), ('f', 7463), ('w', 6302), ('v', 5349), ('z', 2743), ('j', 1861), ('x', 1721), ('q', 976)]
a
Guessing letter: a
6 , 3  : Current masked word: a____a_e

r
Guessing letter: r
5 , 5  : Current masked word: a____i_e_i_
[('n', 538), ('l', 535), ('s', 530), ('t', 432), ('o', 365), ('d', 348), ('c', 336), ('m', 287), ('u', 228), ('b', 197), ('g', 191), ('p', 184), ('h', 179), ('y', 150), ('k', 119), ('v', 105), ('f', 72), ('z', 72), ('w', 67), ('x', 39), ('q', 16), ('j', 15)]
n
Guessing letter: n
5 , 6  : Current masked word: a____ine_i_
[('l', 400), ('s', 369), ('t', 283), ('o', 282), ('d', 259), ('m', 222), ('c', 198), ('u', 187), ('b', 159), ('g', 155), ('y', 133), ('h', 118), ('p', 118), ('k', 102), ('v', 74), ('w', 63), ('f', 61), ('z', 39), ('x', 34), ('j', 14), ('q', 10)]
l
Guessing letter: l
4 , 7  : Current masked word: a____ine_i_
[('s', 369), ('t', 283), ('o', 282), ('d', 259), ('m', 222), ('c', 198), ('u', 187), ('b', 159), ('g', 155), ('y', 133), ('h', 118), ('p', 118), ('k', 102), ('v', 74), ('w', 63), ('f', 61), ('z', 39), ('x', 34), ('j', 14), ('q', 10)]
s
Guessing letter: s
3 , 8  : Current masked word: a____ine_i_
[('t', 28

[('i', 184777), ('a', 179864), ('n', 152290), ('o', 150079), ('r', 149247), ('s', 148475), ('t', 137293), ('l', 122446), ('c', 89382), ('u', 77315), ('d', 74860), ('p', 65798), ('m', 62199), ('h', 58066), ('g', 51859), ('y', 40992), ('b', 39845), ('f', 26434), ('v', 21057), ('k', 18688), ('w', 17734), ('z', 8749), ('x', 6050), ('q', 3987), ('j', 3790)]
i
Guessing letter: i
5 , 3  : Current masked word: _i__________
[('s', 13801), ('n', 11895), ('l', 10985), ('t', 10840), ('r', 10530), ('a', 10158), ('d', 8939), ('o', 8765), ('c', 6777), ('m', 6410), ('g', 5523), ('p', 4691), ('h', 4467), ('b', 3557), ('u', 3493), ('f', 2706), ('y', 2552), ('v', 2294), ('k', 2257), ('w', 1991), ('z', 841), ('x', 382), ('j', 281), ('q', 175)]
s
Guessing letter: s
4 , 4  : Current masked word: _i__________
[('n', 11897), ('l', 10970), ('t', 10842), ('r', 10527), ('a', 10156), ('d', 8942), ('o', 8763), ('c', 6778), ('m', 6419), ('g', 5524), ('p', 4689), ('h', 4466), ('b', 3559), ('u', 3490), ('f', 2708), (

c
Guessing letter: c
4 , 6  : Current masked word: _ar_a_e
[('s', 62), ('g', 59), ('n', 58), ('l', 53), ('m', 52), ('p', 43), ('d', 43), ('b', 42), ('k', 36), ('w', 36), ('i', 31), ('f', 25), ('h', 24), ('v', 24), ('y', 23), ('o', 13), ('u', 12), ('j', 11), ('z', 9), ('x', 3), ('q', 2)]
s
Guessing letter: s
4 , 7  : Current masked word: _arsa_e
[('l', 6), ('p', 5), ('w', 4), ('y', 3), ('b', 3), ('d', 3), ('i', 3), ('m', 3), ('v', 3), ('f', 2), ('g', 2), ('k', 2), ('n', 2), ('o', 2), ('u', 2), ('z', 2), ('h', 1), ('j', 1), ('q', 1), ('x', 1)]
l
Guessing letter: l
3 , 8  : Current masked word: _arsa_e
[('p', 5), ('w', 4), ('y', 3), ('b', 3), ('d', 3), ('i', 3), ('m', 3), ('v', 3), ('f', 2), ('g', 2), ('k', 2), ('n', 2), ('o', 2), ('u', 2), ('z', 2), ('h', 1), ('j', 1), ('q', 1), ('x', 1)]
p
Guessing letter: p
3 , 9  : Current masked word: parsa_e
[('y', 2)]
y
Guessing letter: y
 Successfully Word guessed: parsaye
 Run number :  8  and success :  7
 Run number :  9  and success :  7
Word 

In [44]:
from collections import defaultdict
import random

def build_bigram_model(text):
    """
    Build a bigram language model from a given text.

    Parameters:
    - text (str): Input text.

    Returns:
    - dict: Bigram model with conditional probabilities.
    """
    bigram_model = defaultdict(lambda: defaultdict(int))
    tokens = text.split()

    for i in range(len(tokens) - 1):
        bigram_model[tokens[i]][tokens[i + 1]] += 1

    # Calculate conditional probabilities
    for current_word in bigram_model:
        total_count = sum(bigram_model[current_word].values())
        for next_word in bigram_model[current_word]:
            bigram_model[current_word][next_word] /= total_count

    return bigram_model

def interpolate_words(bigram_model, seed_word, num_interpolations):
    """
    Interpolate words based on a bigram language model.

    Parameters:
    - bigram_model (dict): Bigram language model.
    - seed_word (str): Seed word for interpolation.
    - num_interpolations (int): Number of words to interpolate.

    Returns:
    - list: Interpolated words.
    """
    interpolated_words = []

    current_word = seed_word
    for _ in range(num_interpolations):
        if current_word in bigram_model:
            next_word_options = list(bigram_model[current_word].keys())
            probabilities = list(bigram_model[current_word].values())
            print("current word : ",current_word," with probability : ",probabilities)
            # Choose the next word based on conditional probabilities
            next_word = random.choices(next_word_options, probabilities)[0]
            interpolated_words.append(next_word)

            # Move to the next word
            current_word = next_word
        else:
            # If the current word is not in the model, break the loop
            break

    return interpolated_words

# Example Usage:
if __name__ == "__main__":
    # Example text
    input_text = "the quick brown fox jumps over the lazy dog"

    # Build a bigram model
    bigram_model = build_bigram_model(input_text)
    print(bigram_model)

    # Seed word for interpolation
    seed_word = "quick"

    # Number of words to interpolate
    num_interpolations = 50

    # Interpolate words
    interpolated_words = interpolate_words(bigram_model, seed_word, num_interpolations)

    print(f"Interpolated words starting from '{seed_word}': {interpolated_words}")

defaultdict(<function build_bigram_model.<locals>.<lambda> at 0x000001F83F63CA40>, {'the': defaultdict(<class 'int'>, {'quick': 0.5, 'lazy': 0.5}), 'quick': defaultdict(<class 'int'>, {'brown': 1.0}), 'brown': defaultdict(<class 'int'>, {'fox': 1.0}), 'fox': defaultdict(<class 'int'>, {'jumps': 1.0}), 'jumps': defaultdict(<class 'int'>, {'over': 1.0}), 'over': defaultdict(<class 'int'>, {'the': 1.0}), 'lazy': defaultdict(<class 'int'>, {'dog': 1.0})})
current word :  quick  with probability :  [1.0]
current word :  brown  with probability :  [1.0]
current word :  fox  with probability :  [1.0]
current word :  jumps  with probability :  [1.0]
current word :  over  with probability :  [1.0]
current word :  the  with probability :  [0.5, 0.5]
current word :  lazy  with probability :  [1.0]
Interpolated words starting from 'quick': ['brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']
